## Dependencies

In [17]:
import numpy as np
import pandas as pd

## Core Algorithm

In [32]:
signal1 = np.random.rand(5)
signal2 = np.random.rand(7)

print(signal1.shape,type(signal2))

(5,) <class 'numpy.ndarray'>


In [33]:
def dtw(signal1:np.array,signal2:np.array,normalized:bool=True):

    N,M = len(signal1),len(signal2)

    # calculate the distance between signals
    distance_matrix = np.zeros(shape=(N,M))
    for i in range(N):
        for j in range(M):
            distance_matrix[i,j] = abs(signal1[i]-signal2[j])

    # # Initialize the cos matrix
    cost_matrix = np.full((N+1, M+1), np.inf)
    cost_matrix[0, 0] = distance_matrix[0,0]

    # filling the cost matrix
    traceback = np.zeros(shape=(N,M))
    for i in range(N):
        for j in range(M):
            choices=[
                cost_matrix[i,j],      # match (0)
                cost_matrix[i,j+1],    # insertion (1)
                cost_matrix[i+1,j]     # deletion (2)
            ]

            min_choice = np.argmin(choices)
            cost_matrix[i+1,j+1] = distance_matrix[i,j] + choices[min_choice]

            traceback[i,j]=min_choice

    dtw_distance = cost_matrix[N,M]

    # traceback from bottom right
    i = N-1
    j = M-1
    path = [(i,j)]
    while i>0 or j>0:
        # Handle boundaries explicitly
        if i == 0:
            j -= 1
        elif j == 0:
            i -= 1
        else:
            tb_direction = traceback[i, j]
            if tb_direction == 0:  # diagonal
                i -= 1
                j -= 1
            elif tb_direction == 1:  # up/insertion
                i -= 1
            elif tb_direction == 2:  # left/deletion
                j -= 1
        path.append((i, j))
    path.reverse()

    if normalized:
        return dtw_distance/len(path)
    else:
        return dtw_distance
    


In [35]:
x = dtw(signal1,signal2,normalized=False)
print(x)

1.4394350779823675
